In [1]:
import numpy as np
import tensorflow as tf


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [11]:

# get data from train_1.csv
tmp = np.genfromtxt("train_2.csv", dtype=float, delimiter=",")

# set training_data quantity and validation data quantity
train_quantity = 800
validation_quantity = 891-train_quantity

# 归一化
tem = tmp[1:,3:]
ave = np.mean(tem,axis = 0)*np.ones((891,7))
ptp = np.ptp(tem,axis = 0)*np.ones((891,7))
tem = np.subtract(tem,ave)
tem = np.true_divide(tem,ptp)
tmp[1:,3:] = tem

#apart train,validation
train_data = tmp[1:train_quantity+1,3:]
validation_data0 = tmp[train_quantity+1:,3:]

train_lable = tmp[1:train_quantity+1,2]
validation_lable0 = tmp[train_quantity+1:,2]

train_lable = train_lable.reshape(train_quantity,1)
validation_lable0 = validation_lable0.reshape(validation_quantity,1)

# define layer function
def add_layer(input,in_size,out_size):
    theta = tf.Variable(tf.random_normal([in_size,out_size]))
    if out_size==1:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
    else:
        z = tf.matmul(input,theta)
        a = tf.nn.relu(z)
        a_0 = tf.ones([train_quantity,1])
        a = tf.concat([a_0,a],1)
    return theta,a,z


# define placeholder for inputs to network
xs = tf.placeholder(tf.float32,[None,7])
ys = tf.placeholder(tf.float32,[None,1])

# add hidden layer
theta_1,l1,z_1 = add_layer(xs,7,8)
# add output layer
theta_2,output,z_2 = add_layer(l1,9,1)

#regularzation
lamda = 0.09
regularzation = (tf.reduce_sum(tf.square(theta_1))+tf.reduce_sum(tf.square(theta_2)))*lamda/600

# the error between prediction and real data
J_theta = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = ys,logits = z_2)) + regularzation

a =  0.5
# learning rate
train_step = tf.train.GradientDescentOptimizer(a).minimize(J_theta)

#prediction
validation_data = tf.constant(validation_data0,tf.float32)
validation_lable = tf.constant(validation_lable0,tf.float32)

a_1 = tf.nn.relu(tf.matmul(validation_data,theta_1))
a_0 = tf.ones([validation_quantity,1])
a_20 = tf.concat([a_0,a_1],1)
a_2 = tf.sigmoid(tf.matmul(a_20,theta_2))
prediction = tf.round(a_2)

validation = tf.subtract(prediction,validation_lable)
validation = tf.abs(validation)
accuracy =tf.reduce_sum(validation) 
accuracy = (validation_quantity-accuracy)/validation_quantity

#initialize
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(40000):
    #training
    sess.run(train_step,feed_dict={xs:train_data,ys:train_lable})
    
    if i%500 == 0:
        loss = sess.run(J_theta,feed_dict={xs:train_data,ys:train_lable})
        print(i)
        print("J_theta:",loss)
        acc = sess.run(accuracy)
        print(acc)
        print("\n")




0
J_theta: 0.8719268
0.62637365


500
J_theta: 0.43149027
0.82417583


1000
J_theta: 0.4251858
0.84615386


1500
J_theta: 0.4222822
0.85714287


2000
J_theta: 0.41999555
0.85714287


2500
J_theta: 0.4179171
0.84615386


3000
J_theta: 0.41524652
0.84615386


3500
J_theta: 0.4140807
0.84615386


4000
J_theta: 0.41319045
0.84615386


4500
J_theta: 0.41243088
0.84615386


5000
J_theta: 0.41117024
0.84615386


5500
J_theta: 0.41050312
0.84615386


6000
J_theta: 0.40996817
0.84615386


6500
J_theta: 0.40961212
0.84615386


7000
J_theta: 0.40923682
0.84615386


7500
J_theta: 0.40903077
0.84615386


8000
J_theta: 0.4085883
0.84615386


8500
J_theta: 0.40841126
0.84615386


9000
J_theta: 0.40827307
0.84615386


9500
J_theta: 0.40819952
0.84615386


10000
J_theta: 0.40808865
0.84615386


10500
J_theta: 0.40801585
0.84615386


11000
J_theta: 0.40792122
0.84615386


11500
J_theta: 0.40787023
0.84615386


12000
J_theta: 0.4078253
0.84615386


12500
J_theta: 0.4077806
0.84615386


13000
J_theta: 0.4

In [3]:
# import test sets
test = np.genfromtxt("test_2.csv",dtype = float,delimiter=",")

#reshape test sets
tem1 = test[1:,2:]
ave1 = np.mean(tem1,axis = 0)*np.ones((418,7))
ptp1 = np.ptp(tem1,axis = 0)*np.ones((418,7))
tem1 = np.subtract(tem1,ave1)
tem1 = np.true_divide(tem1,ptp1)
test[1:,2:] = tem1
test = test[1:,2:]

# define tensorflow constant test data
test_data = tf.constant(test,tf.float32)

# predict test data
t_1 = tf.nn.relu(tf.matmul(test_data,theta_1))
t_0 = tf.ones([418,1])
t_20 = tf.concat([t_0,t_1],1)
t_2 = tf.sigmoid(tf.matmul(t_20,theta_2))
prediction_1 = tf.round(t_2)
prediction_1 = tf.to_int32(prediction_1)

# return result
result = sess.run(prediction_1)
No = np.arange(892,1310)
No = No.reshape(418,1)
title = np.array(["PassengerId","Survived"])
title = title.reshape(1,2)
result = np.append(No,result,axis=1)
result = np.append(title,result,axis=0)
np.savetxt("result.csv", result, delimiter=",",fmt = "%s")


In [4]:
sess.run(t_2)

array([[0.06973022],
       [0.35025236],
       [0.07577372],
       [0.09820674],
       [0.33200297],
       [0.21584734],
       [0.68748695],
       [0.11021296],
       [0.7363166 ],
       [0.04166181],
       [0.09013495],
       [0.31801787],
       [0.97284085],
       [0.04029924],
       [0.9923171 ],
       [0.9725884 ],
       [0.09532345],
       [0.32590806],
       [0.31124038],
       [0.72130257],
       [0.32884344],
       [0.38859636],
       [0.9823254 ],
       [0.7185731 ],
       [0.9726998 ],
       [0.03056204],
       [0.9946404 ],
       [0.28917477],
       [0.34100965],
       [0.1257025 ],
       [0.05797081],
       [0.13843544],
       [0.201969  ],
       [0.21994197],
       [0.50090516],
       [0.39197904],
       [0.353291  ],
       [0.43693602],
       [0.10419726],
       [0.12873214],
       [0.06948075],
       [0.41642538],
       [0.06687157],
       [0.860982  ],
       [0.991273  ],
       [0.10296143],
       [0.27777204],
       [0.072